# Environment Setup & Load data

In [ ]:
import json
import re
import nltk
import spacy
nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')
import nltk.data
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
sentences = []
with open('anlp-sciner-test.txt', 'r',encoding='utf-8') as f:
  for line in f:
      sentences.append(line)

      # save sentences with keywords in json format
json_object = json.dumps(sentences, indent=4)
with open("test_sentences.json", "w") as outfile:
    outfile.write(json_object)



# BIO Sequence Labeling

In [ ]:
features_list = []
with open('word_dict.txt','r',encoding='utf-8') as f:
    for line in f.readlines():
        features_list.append(line.strip().split(',')[0])


label_dict={}
with open('word_dict.txt','r',encoding='utf-8') as f:
     for line in f.readlines():
          item = line.split(',')
          label_dict[item[0].strip()]=item[1].strip()

In [ ]:
def BIO_labeling(inputs,output='output'):
    file_output = output+'.conll'
    index_log = 0
    writen_flag=0

    for line in inputs:
        # print(line)
        word_list = [t.text.strip() for t in nlp(line.strip())]
        tag_list = ["O" for i in range(len(word_list))]
    

        for keyword in features_list:
            metric_flag = 0    # Check if there's metricName or hyperparameter in the sentence
            index_start_tag=0
            index_log = 0
            # print('Keyword: ',keyword)

            while 1:
                remaining_sentence = ' '.join(word_list[index_log:])
                if keyword not in remaining_sentence:    # keyword does not exist in the remaining part of the sentence.
                    index_log = 0
                    break

                # keyword exist in the remaining part of the sentence.----------------------
                search_word = keyword
                for tk in word_list[index_log:]:
                  if keyword in tk:    # keyword exist as a substring of one of tokens. Example BERT in VideoBERT
                    search_word = tk
                    break
                
                search_length = len(search_word.split(' '))
                if search_length == 1:        # If search word contains only one word
                    index_start_tag = word_list.index(search_word,index_log)    # locate the keyword        
                    index_log = index_start_tag+1                           # set the index to the next word after the located keyword 
                else:                         # If search word contains more than one word
                    try:
                      index_start_tag = word_list.index(search_word.split(' ')[0],index_log)           
                      index_log = index_start_tag+search_length                       
                    except ValueError: break

                # BIO labeling according to the length of the keyword and position pointer
                for i in range(index_start_tag, index_start_tag + search_length):
                    if index_start_tag == i:                # beginning of the label, mark with B-

                        
                        if tag_list[i] == 'O':
                            tag_list[i] = "B-"+label_dict[keyword].replace("\n",'')

                    else:
                        if tag_list[i] == 'O':
                            tag_list[i] = "I-"+label_dict[keyword].replace("\n",'')  # middle of the label, mark with I-

                
                
                # labeling value 
                if search_word in label_dict.keys():
                  if label_dict[search_word]=='HyperparameterName' or label_dict[search_word]=='MetricName':  
                    metric_flag=1
                    # print('Aha! Found {}!'.format(label_dict[search_word]))
                

                for tk in word_list[index_log-1:]:
                  if metric_flag:    # metricName or hyperparameterName exist in the sentence, we detect pure value(integer or float) in the sentence as well.
                    if re.match('^\d*(\.\d+)?$',tk):
                      try:
                        label_name = label_dict[search_word].split('N')[0]+'Value'    # get label name: MetricValue or HyperparameterValue
                        label_position = word_list.index(tk)
                        if tag_list[label_position] == 'O':
                          tag_list[label_position] = "B-"+label_name
                        break
                      except KeyError: continue

        with open(file_output,'a',encoding='utf-8') as output_f:
            if writen_flag == 1:
                output_f.write('\n')
            for w,t in zip(word_list,tag_list):
                      if w != '	' and t != ' ':
                          output_f.write(w+" "+t+'\n')
                          writen_flag=1

        

In [ ]:
# BIO_labeling(all_sentences,'training_data')

BIO_labeling(all_sentences,'test_with_labels')

#BIO_labeling(all_sentences,'test_yubo')
